In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib
import pickle
import random
import math
import CRPS.CRPS as pscore
import datetime

pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)


import sys
sys.path.append('../../TaskExecutionTimeMining/')
from event_log_transformer import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from normal_evaluation.drbart_evaluation import *

In [2]:
with open('../../../data/BPIC2017/transformed_event_logs/BPIC2017_train.pickle', 'rb') as f:
    train_data = pickle.load(f)

with open('../../../data/BPIC2017/transformed_event_logs/BPIC2017_test.pickle', 'rb') as f:
    test_data = pickle.load(f)

In [3]:
train_data

,Action_start,org:resource_start,concept:name,EventOrigin_start,EventID_start,lifecycle:transition_start,time:timestamp_start,case:LoanGoal_start,case:ApplicationType_start,case:concept:name,case:RequestedAmount_start,Action_complete,org:resource_complete,EventOrigin_complete,EventID_complete,lifecycle:transition_complete,time:timestamp_complete,case:LoanGoal_complete,case:ApplicationType_complete,case:RequestedAmount_complete,duration,duration_seconds,duration_ms,duration_hours,seconds_in_day,day_of_week,User_1,User_10,User_100,User_101,User_102,User_103,User_104,User_105,User_106,User_107,User_108,User_109,User_11,User_110,User_111,User_112,User_113,User_114,User_115,User_116,User_117,User_118,User_119,User_12,User_120,User_121,User_122,User_123,User_124,User_125,User_126,User_127,User_128,User_129,User_13,User_130,User_131,User_132,User_133,User_134,User_135,User_136,User_137,User_138,User_139,User_14,User_140,User_141,User_142,User_143,User_144,User_145,User_146,User_147,User_148,User_149,User_15,User_16,User_17,User_18,User_19,User_2,User_20,User_21,User_22,User_23,User_24,User_25,User_26,User_27,User_28,User_29,User_3,User_30,User_31,User_32,User_33,User_34,User_35,User_36,User_37,User_38,User_39,User_4,User_40,User_41,User_42,User_43,User_44,User_45,User_46,User_47,User_48,User_49,User_5,User_50,User_51,User_52,User_53,User_54,User_55,User_56,User_57,User_58,User_59,User_6,User_60,User_61,User_62,User_63,User_64,User_65,User_66,User_67,User_68,User_69,User_7,User_70,User_71,User_72,User_73,User_74,User_75,User_76,User_77,User_78,User_79,User_8,User_80,User_81,User_82,User_83,User_84,User_85,User_86,User_87,User_88,User_89,User_9,User_90,User_91,User_92,User_93,User_94,User_95,User_96,User_97,User_98,User_99,W_Assess_potential_fraud,W_Call_after_offers,W_Call_incomplete_files,W_Complete_application,W_Handle_leads,W_Shortened_completion_,W_Validate_application
1191520,Created,User_112,W_Validate application,Workflow,Workitem_100000116,schedule,2016-03-02 13:25:52.388000+00:00,Home improvement,Limit raise,Application_1650545424,40000.0,Obtained,User_112,Workflow,Workitem_685532007,start,2016-03-02 13:25:52.390000+00:00,Home improvement,Limit raise,40000.0,0 days 00:00:00.002000,0.002,2.000000e+00,5.555556e-07,48352,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5344983,Released,User_63,W_Call after offers,Workflow,Workitem_1000010198,suspend,2016-08-20 12:19:22.434000+00:00,Home improvement,New credit,Application_1930272371,22500.0,Deleted,User_1,Workflow,Workitem_83701135,ate_abort,2016-09-16 06:00:42.189000+00:00,Home improvement,New credit,22500.0,26 days 17:41:19.755000,2310079.755,2.310080e+09,6.416888e+02,44362,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
6776886,Obtained,User_80,W_Call after offers,Workflow,Workitem_1000013868,start,2016-10-18 08:54:52.604000+00:00,Car,New credit,Application_1804686886,6500.0,Released,User_80,Workflow,Workitem_282145227,suspend,2016-10-18 08:57:35.834000+00:00,Car,New credit,6500.0,0 days 00:02:43.230000,163.230,1.632300e+05,4.534167e-02,32092,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
7210692,Released,User_30,W_Call incomplete files,Workflow,Workitem_1000014801,suspend,2016-11-30 16:55:49.4

In [ ]:
import sys
sys.path.append('../../TaskExecutionTimeMining/pix-framework-main/src')

from dumas_model import DumasModel

dm = DumasModel(train_data, resource=True,
               concept_name = 'concept:name',
               resource_name = 'org:resource_start')

dm.set_up_models()

with open('./dumas_model.pickle', 'wb') as handle:
    pickle.dump(dm, handle)

In [ ]:
dm = DumasModel(train_data, resource=False,
               concept_name = 'concept:name',
               resource_name = 'org:resource_start')
dm.set_up_models()

with open('./dumas_model_no_resource.pickle', 'wb') as handle:
    pickle.dump(dm, handle)